In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [970]:
df = pd.read_excel('Fantasy_ML.xlsx', sheetname='Sheet1')
df.head()

/home/addybrown/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


,Player,Year,PER,BPM,VORP,OWS,DWS,WS,Fantasy_Pts
0,Mark Acres,1990,8.3,-2.0,0.0,1.1,0.6,1.6,962.75
1,Michael Adams,1990,15.4,1.7,2.5,4.4,2.5,6.9,2554.25
2,Mark Aguirre,1990,15.8,0.6,1.3,3.1,2.5,5.7,1653.00
3,Danny Ainge,1990,16.1,1.6,2.5,2.7,2.1,4.8,2593.75
4,Mark Alarie,1990,14.1,-1.0,0.5,1.5,1.6,3.1,1591.25


In [971]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [972]:
x=df.drop(['Fantasy_Pts','Player','Year'],axis=1)
y=df['Fantasy_Pts']

In [973]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [974]:
scale = StandardScaler()
x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
y_train=y_train.reshape(12412,1)
x_test=np.asarray(x_test)
y_test=np.asarray(y_test)
y_test=y_test.reshape(5320,1)

In [975]:
import tensorflow as tf

In [976]:
# Defining input nodes, hidden layers and output 
n_input=7
n_hidden=100
n_output=1
n_samples=x_train.shape[0]

In [977]:
learning_rate = 0.01
training_epochs = 1000
display_step = 50

In [978]:
X=tf.placeholder(tf.float32,shape=[12412, 7])
Y=tf.placeholder(tf.float32,shape=[12412,1])

In [979]:
rng = np.random
#Weights
W1 = tf.Variable(tf.random_uniform([n_input,n_hidden]), name="W1")
W2 = tf.Variable(tf.random_uniform([n_hidden,n_output]), name="W2")
#Bias
b1 = tf.Variable(tf.zeros([12412,n_hidden]), name="Bias1")
b2 = tf.Variable(tf.zeros([12412,1]), name="Bias2")
                

In [980]:
#Computation
L2 = tf.add(tf.matmul(X, W1), b1)
hy= tf.add(tf.matmul(L2,W2),b2)

In [981]:
#Cost Function
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=hy))

In [982]:
#Optimization
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


In [983]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


In [984]:
with tf.Session() as sess:
    sess.run(init)

    for step in range(30):
        _, c = sess.run([optimizer, cost], feed_dict = {X: x_train, Y: y_train})

        if step % display_step == 0:
            print("Cost: ", c)

    answer = tf.equal(tf.floor(hy + 0.1), Y)
    accuracy = tf.reduce_mean(tf.cast(answer, tf.float32))

    print(sess.run([hy], feed_dict = {X: x_train, Y: y_train}))
        

ValueError: Cannot feed value of shape (12412, 6) for Tensor 'Placeholder_66:0', which has shape '(12412, 7)'

In [1295]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.3)

In [1296]:
feat_cols=[]

for col in x.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [1297]:
feat_cols

[_NumericColumn(key='PER', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='BPM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='VORP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='OWS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='DWS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='WS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [1298]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train1,y=y_train1,batch_size=10000,num_epochs=100000,shuffle=True)

In [1299]:
Regression = tf.estimator.DNNRegressor(hidden_units=[200, 400,200],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpiv2dp1am', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa0e464edd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [1300]:
Regression.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpiv2dp1am/model.ckpt.
INFO:tensorflow:loss = 15350028000.0, step = 1
INFO:tensorflow:global_step/sec: 2.72583
INFO:tensorflow:loss = 1587391200.0, step = 101 (36.687 sec)
INFO:tensorflow:global_step/sec: 2.88418
INFO:tensorflow:loss = 1469388200.0, step = 201 (34.673 sec)
INFO:tensorflow:global_step/sec: 2.4719
INFO:tensorflow:loss = 1401890700.0, step = 301 (40.453 sec)
INFO:tensorflow:global_step/sec: 2.86126
INFO:tensorflow:loss = 1337745700.0, step = 401 (34.950 sec)
INFO:tensorflow:global_step/sec: 2.48364
INFO:tensorflow:loss = 1305306500.0, step = 501 (40.264 sec)
INFO:tensorflow:global_step/sec: 2.8773
INFO:tensorflow:loss = 1309853700.0, step = 601 (34.755 sec)
INFO:tensorflow:global_step/sec

In [1301]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test1,batch_size=len(x_test1),shuffle=False)

In [1302]:
note_predictions = list(Regression.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiv2dp1am/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [1303]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['predictions'][0])

In [1304]:
from sklearn.metrics import classification_report,confusion_matrix

In [1305]:
#Approximate Performance
np.mean(abs(y_test1-final_preds))

230.06089089427675

In [1306]:
df5=pd.DataFrame(y_test1)
df5.reset_index()
df5['index']=final_preds
df5['Difference']=abs(df5['Fantasy_Pts']-df5['index'])
j=len(df5[df5['Difference']<250])
l=len(df5)

In [1307]:
j/l

0.7133458646616542

In [1308]:
#Import Sample 2017-2018 to inspect model

sample=pd.read_excel('2017_2018_Sample.xlsx', sheetname='Sheet1')

/home/addybrown/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [1309]:
sample.columns=['Rk', 'Player', 'Pos', 'Age ', 'Tm ', 'G ', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr ', 'ORB% ', 'DRB% ', 'TRB% ', 'AST%', 'STL% ', 'BLK%',
       'TOV%', 'USG% ', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']
sample=sample[['Player','PER', 'BPM', 'VORP', 'OWS', 'DWS', 'WS']]
sample.head()

,Player,PER,BPM,VORP,OWS,DWS,WS
0,James Harden,29.8,10.9,8.3,11.6,3.8,15.4
1,LeBron James,28.6,9.6,8.9,11.0,3.0,14.0
2,Karl-Anthony Towns,24.9,5.5,5.5,10.6,3.4,14.0
3,Anthony Davis,28.9,5.2,4.9,8.8,4.9,13.7
4,Damian Lillard,25.2,6.7,5.9,9.9,2.7,12.6


In [1310]:
sample=sample.groupby('Player').sum()
sample=sample.sort_values('WS',ascending=False)
sample.head()

,PER,BPM,VORP,OWS,DWS,WS
Player,,,,,,
James Harden,29.8,10.9,8.3,11.6,3.8,15.4
LeBron James,28.6,9.6,8.9,11.0,3.0,14.0
Karl-Anthony Towns,24.9,5.5,5.5,10.6,3.4,14.0
Anthony Davis,28.9,5.2,4.9,8.8,4.9,13.7
Damian Lillard,25.2,6.7,5.9,9.9,2.7,12.6


In [1311]:
pred_fn_sample = tf.estimator.inputs.pandas_input_fn(x=sample,batch_size=len(sample),shuffle=False)

In [1312]:
note_predictions_sample = list(Regression.predict(input_fn=pred_fn_sample))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiv2dp1am/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [1313]:
final_preds_sample  = []
for pred in note_predictions_sample:
    final_preds_sample.append(pred['predictions'][0])

In [1314]:
predict_2017_2018=pd.DataFrame(final_preds_sample,index=sample.index.values)
predict_2017_2018.head()

,0
James Harden,3712.479492
LeBron James,4288.428711
Karl-Anthony Towns,3467.526123
Anthony Davis,3566.430420
Damian Lillard,3295.810059


In [1315]:
predict_2017_2018.columns=['Fantasy_Pts']
predict_2017_2018.sort_values('Fantasy_Pts',ascending=False).head()

,Fantasy_Pts
LeBron James,4288.428711
Russell Westbrook,3922.975342
James Harden,3712.479492
Anthony Davis,3566.430420
Giannis Antetokounmpo,3493.288574


In [1316]:
'''For Some reason the model is more accurate when adding 500 points to it which is kind of strange
but the point is it, was accurate at calculating the rankings which will ultimately be more important'''

predict_2017_2018['Fantasy_Pts']=predict_2017_2018['Fantasy_Pts']+500

In [1317]:
'''
POST 2017/2018 PREDICTION HERE AND COMPARE '''


'\nPOST 2017/2018 PREDICTION HERE AND COMPARE '

In [1318]:
dg=pd.read_csv('2017_2018_Full.csv')

In [1319]:
#Creating a storage of all 2017_2018 Fantasy Data
dg=dg[['Player?','PER', 'BPM', 'VORP', 'OWS', 'DWS', 'WS']]
dg = dg.rename(columns={'Player?': 'Player'})
dg['Player']=dg.Player.apply(lambda x: x[:x.find("\\")])
dg.head()

,Player,PER,BPM,VORP,OWS,DWS,WS
0,Alex Abrines,9.0,-2.2,-0.1,1.3,1.0,2.2
1,Quincy Acy,8.2,-2.2,-0.1,-0.1,1.1,1.0
2,Steven Adams,20.6,3.3,3.3,6.7,3.0,9.7
3,Bam Adebayo,15.7,0.2,0.8,2.3,1.9,4.2
4,Arron Afflalo,5.8,-5.8,-0.7,-0.1,0.2,0.1


In [1320]:
''' Using ThirtyFiveEight's Carmelo 
Projections on BPM for the 2018-2019 Season and using a seperate Machine Learning algorithm relating it to the other 
advanced stats a prediction of 2018-2019 can be made'''

" Using ThirtyFiveEight's Carmelo \nProjections on BPM for the 2018-2019 Season and using a seperate Machine Learning algorithm relating it to the other \nadvanced stats a prediction of 2018-2019 can be made"

In [1321]:
stats_2018_2019=df = pd.read_excel('2018_2019_BoxPlus_Minus.xlsx', sheetname='Sheet1')

/home/addybrown/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [1322]:
Merged_Data=pd.merge(dg,stats_2018_2019,how='left',left_on='Player',right_on='Player')

In [1323]:
Merged_Data=Merged_Data[pd.isnull(Merged_Data['Off(+)'])==False]

In [1324]:
Merged_Data['BPM']=Merged_Data['Agg']

In [1325]:
Merged_Data=Merged_Data.drop(['Off(+)','Def(+)','Agg'],axis=1)

In [1326]:
Merged_Data=Merged_Data.groupby('Player').mean()

In [1327]:
pred_fn_merged = tf.estimator.inputs.pandas_input_fn(x=Merged_Data,batch_size=len(Merged_Data),shuffle=False)

In [1328]:
note_predictions_merged = list(Regression.predict(input_fn=pred_fn_merged))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiv2dp1am/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [1329]:
final_preds_merged  = []
for pred in note_predictions_merged:
    final_preds_merged.append(pred['predictions'][0])

In [1335]:
predict_2018_2019=pd.DataFrame(final_preds_merged,index=Merged_Data.index.values)
predict_2018_2019.head()
predict_2018_2019.columns=['Fantasy_Pts']
predict_2018_2019.sort_values('Fantasy_Pts',ascending=False).head()

,Fantasy_Pts
LeBron James,5757.749512
James Harden,5289.175293
Andre Drummond,4518.536133
Damian Lillard,4498.370605
Russell Westbrook,4277.670898


In [1337]:
#Final Predictions for 2018/2019 NBA Season 
predict_2018_2019=predict_2018_2019.sort_values('Fantasy_Pts',ascending=False)
predict_2018_2019.head()

,Fantasy_Pts
LeBron James,5757.749512
James Harden,5289.175293
Andre Drummond,4518.536133
Damian Lillard,4498.370605
Russell Westbrook,4277.670898


In [1338]:
writer = pd.ExcelWriter('Predictions2018_2019.xlsx')
predict_2018_2019.to_excel(writer,'Sheet1')
writer.save()